In [ ]:
!pip install optuna

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import fbeta_score
import optuna
import joblib
from imblearn.under_sampling import RandomUnderSampler

# Read data
data = pd.read_excel('TrainClass.xlsx')
real_data = pd.read_excel('TestClass.xlsx')

# Prepare training data
Y, X = data['FRACASO'], data.drop(['FRACASO', 'CODIGO_EMPRESA'], axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=5)

# Prepare testing data
X = real_data.drop(['CODIGO_EMPRESA'], axis=1)

# Define the objective function for BaggingClassifier with RandomForestClassifier
def bagging_rf_objective(trial):
    # Define hyperparameters for RandomForestClassifier
    rf_n_estimators = trial.suggest_int('rf_n_estimators', 50, 200)
    rf_max_depth = trial.suggest_int('rf_max_depth', 1, 10)

    # Define hyperparameters for BaggingClassifier
    bagging_n_estimators = trial.suggest_int('bagging_n_estimators', 10, 100)
    bagging_max_samples = trial.suggest_float('bagging_max_samples', 0.1, 1.0)

    # Create the base classifier with suggested hyperparameters
    base_classifier = RandomForestClassifier(n_estimators=rf_n_estimators, max_depth=rf_max_depth, random_state=0)

    # Create the bagging classifier with the base classifier
    bagging_classifier = BaggingClassifier(
        base_classifier,
        n_estimators=bagging_n_estimators,
        max_samples=bagging_max_samples,
        random_state=0
    )

    # Use RandomUnderSampler to undersample the majority class
    under_sampler = RandomUnderSampler(sampling_strategy='auto', random_state=0)
    X_resampled, Y_resampled = under_sampler.fit_resample(X_train, Y_train)

    # Define cross-validation strategy (StratifiedKFold for classification)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

    # Perform cross-validation and get mean F-beta score
    fbeta_scores = cross_val_score(bagging_classifier, X_resampled, Y_resampled, cv=cv, scoring='f1_weighted')
    mean_fbeta = fbeta_scores.mean()

    return mean_fbeta

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(bagging_rf_objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params
print(f'Best Hyperparameters for BaggingClassifier + RandomForestClassifier: {best_params}')

# Create the base classifier with the best hyperparameters for RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=best_params['rf_n_estimators'], max_depth=best_params['rf_max_depth'], random_state=0)

# Create the bagging classifier with the best hyperparameters
bagging_classifier = BaggingClassifier(
    rf_classifier,
    n_estimators=best_params['bagging_n_estimators'],
    max_samples=best_params['bagging_max_samples'],
    random_state=0
)

# Train the bagging classifier on the resampled data
bagging_classifier.fit(X_train, Y_train)

# Predictions on testing set
y_pred_test = bagging_classifier.predict(X_test)

# Calculate F-beta score for testing set
fbeta_test = fbeta_score(Y_test, y_pred_test, beta=2, average='weighted')
print(f'F-beta Score for Testing Set: {fbeta_test:.4f}')

# Save the trained model to a file
joblib.dump(bagging_classifier, 'bagging_rf_model.joblib')

# Load model and generate probability CSV
loaded_model = joblib.load('bagging_rf_model.joblib')
prob = loaded_model.predict_proba(X)[:, 1]
prob_df = pd.DataFrame(prob, columns=['Probability'])
prob_df.index = prob_df.index + 1
prob_df.index.name = 'Id'
prob_df.to_csv('intento.csv', index=True)


[I 2023-12-06 22:42:57,732] A new study created in memory with name: no-name-0cf86c4d-a52a-48c6-97b0-90c94ffb276f
[I 2023-12-06 22:43:41,475] Trial 0 finished with value: 0.7601364522417154 and parameters: {'rf_n_estimators': 63, 'rf_max_depth': 4, 'bagging_n_estimators': 84, 'bagging_max_samples': 0.7699794792202498}. Best is trial 0 with value: 0.7601364522417154.
[I 2023-12-06 22:45:13,340] Trial 1 finished with value: 0.7601364522417154 and parameters: {'rf_n_estimators': 146, 'rf_max_depth': 8, 'bagging_n_estimators': 76, 'bagging_max_samples': 0.7698456927160355}. Best is trial 0 with value: 0.7601364522417154.
[I 2023-12-06 22:45:59,427] Trial 2 finished with value: 0.7601364522417154 and parameters: {'rf_n_estimators': 152, 'rf_max_depth': 8, 'bagging_n_estimators': 37, 'bagging_max_samples': 0.6599684657271322}. Best is trial 0 with value: 0.7601364522417154.
[I 2023-12-06 22:47:08,967] Trial 3 finished with value: 0.7504394825447457 and parameters: {'rf_n_estimators': 126, 'r

Best Hyperparameters for BaggingClassifier + RandomForestClassifier: {'rf_n_estimators': 63, 'rf_max_depth': 4, 'bagging_n_estimators': 84, 'bagging_max_samples': 0.7699794792202498}
F-beta Score for Testing Set: 0.9441
